In [22]:
import requests
import pandas as pd
import StringIO
import datetime
import json
from urllib import urlencode
import time

import sys
stdin, stdout, stderr = sys.stdin, sys.stdout, sys.stderr
reload(sys)
sys.stdin, sys.stdout, sys.stderr = stdin, stdout, stderr
sys.setdefaultencoding('utf8')

### Выгрузка из Logs API
#### Проверка

In [3]:
TOKEN = '<token>' # получить токен тут - https://oauth.yandex.ru/authorize?response_type=token&client_id=44cc1b35c0ff47db957ccc61624b4e94

In [4]:
API_HOST = 'https://api-metrika.yandex.ru'
COUNTER_ID = 161451
START_DATE = '2019-02-20'
END_DATE = '2019-02-20'
SOURCE = 'hits'
API_FIELDS = ('ym:pv:date', 'ym:pv:dateTime', 'ym:pv:URL', 'ym:pv:deviceCategory', 
         'ym:pv:operatingSystemRoot', 'ym:pv:clientID', 'ym:pv:browser', 'ym:pv:lastTrafficSource',
         'ym:pv:params', 'ym:pv:link', 'ym:pv:download', 'ym:pv:notBounce')

In [13]:
url_params = urlencode(
    [
        ('date1', START_DATE),
        ('date2', END_DATE),
        ('source', SOURCE),
        ('fields', ','.join(API_FIELDS))
    ]
)

headers = {'Authorization': 'OAuth ' + TOKEN}

url = '{host}/management/v1/counter/{counter_id}/logrequests/evaluate?'\
    .format(host=API_HOST, counter_id=COUNTER_ID) + url_params

r = requests.get(url, headers = headers)

/home/miptgirl/jupyterenv/local/lib/python2.7/site-packages/urllib3/util/ssl_.py:160: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. You can upgrade to a newer version of Python to solve this. For more information, see https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecurePlatformWarning


In [14]:
r.status_code

200

In [16]:
json.loads(r.text)['log_request_evaluation']

{u'max_possible_day_quantity': 11, u'possible': True}

#### Создаем запрос

In [17]:
url_params = urlencode(
    [
        ('date1', START_DATE),
        ('date2', END_DATE),
        ('source', SOURCE),
        ('fields', ','.join(sorted(API_FIELDS, key=lambda s: s.lower())))    
    ]
)
url = '{host}/management/v1/counter/{counter_id}/logrequests?'\
    .format(host=API_HOST,
            counter_id=COUNTER_ID) \
      + url_params

r = requests.post(url, headers = headers)

/home/miptgirl/jupyterenv/local/lib/python2.7/site-packages/urllib3/util/ssl_.py:160: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. You can upgrade to a newer version of Python to solve this. For more information, see https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecurePlatformWarning


In [18]:
r.status_code

200

In [19]:
json.loads(r.text)['log_request']

{u'counter_id': 161451,
 u'date1': u'2019-02-20',
 u'date2': u'2019-02-20',
 u'fields': [u'ym:pv:browser',
  u'ym:pv:clientID',
  u'ym:pv:date',
  u'ym:pv:dateTime',
  u'ym:pv:deviceCategory',
  u'ym:pv:download',
  u'ym:pv:lastTrafficSource',
  u'ym:pv:link',
  u'ym:pv:notBounce',
  u'ym:pv:operatingSystemRoot',
  u'ym:pv:params',
  u'ym:pv:URL'],
 u'request_id': 3051382,
 u'source': u'hits',
 u'status': u'created'}

In [20]:
request_id = json.loads(r.text)['log_request']['request_id']

### Ждем окончание обработки

In [23]:
status = 'created'
while status == 'created':
    time.sleep(60)
    print 'trying'
    url = '{host}/management/v1/counter/{counter_id}/logrequest/{request_id}' \
            .format(request_id=request_id,
                    counter_id=COUNTER_ID,
                    host=API_HOST)

    r = requests.get(url, headers =  headers)
    if r.status_code == 200:
        status = json.loads(r.text)['log_request']['status']
        print json.dumps(json.loads(r.text)['log_request'], indent = 4)
    else:
        raise ValueError, r.text

trying
{
    "date1": "2019-02-20", 
    "status": "processed", 
    "date2": "2019-02-20", 
    "counter_id": 161451, 
    "fields": [
        "ym:pv:browser", 
        "ym:pv:clientID", 
        "ym:pv:date", 
        "ym:pv:dateTime", 
        "ym:pv:deviceCategory", 
        "ym:pv:download", 
        "ym:pv:lastTrafficSource", 
        "ym:pv:link", 
        "ym:pv:notBounce", 
        "ym:pv:operatingSystemRoot", 
        "ym:pv:params", 
        "ym:pv:URL"
    ], 
    "source": "hits", 
    "parts": [
        {
            "part_number": 0, 
            "size": 42392753
        }, 
        {
            "part_number": 1, 
            "size": 42357236
        }, 
        {
            "part_number": 2, 
            "size": 41933358
        }, 
        {
            "part_number": 3, 
            "size": 41855947
        }, 
        {
            "part_number": 4, 
            "size": 42363503
        }, 
        {
            "part_number": 5, 
            "size": 42214217
     

/home/miptgirl/jupyterenv/local/lib/python2.7/site-packages/urllib3/util/ssl_.py:160: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. You can upgrade to a newer version of Python to solve this. For more information, see https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecurePlatformWarning


In [24]:
json.loads(r.text)['log_request']

{u'counter_id': 161451,
 u'date1': u'2019-02-20',
 u'date2': u'2019-02-20',
 u'fields': [u'ym:pv:browser',
  u'ym:pv:clientID',
  u'ym:pv:date',
  u'ym:pv:dateTime',
  u'ym:pv:deviceCategory',
  u'ym:pv:download',
  u'ym:pv:lastTrafficSource',
  u'ym:pv:link',
  u'ym:pv:notBounce',
  u'ym:pv:operatingSystemRoot',
  u'ym:pv:params',
  u'ym:pv:URL'],
 u'parts': [{u'part_number': 0, u'size': 42392753},
  {u'part_number': 1, u'size': 42357236},
  {u'part_number': 2, u'size': 41933358},
  {u'part_number': 3, u'size': 41855947},
  {u'part_number': 4, u'size': 42363503},
  {u'part_number': 5, u'size': 42214217},
  {u'part_number': 6, u'size': 41872731},
  {u'part_number': 7, u'size': 42391120}],
 u'request_id': 3051382,
 u'size': 337380865,
 u'source': u'hits',
 u'status': u'processed'}

In [25]:
parts = json.loads(r.text)['log_request']['parts']
parts

[{u'part_number': 0, u'size': 42392753},
 {u'part_number': 1, u'size': 42357236},
 {u'part_number': 2, u'size': 41933358},
 {u'part_number': 3, u'size': 41855947},
 {u'part_number': 4, u'size': 42363503},
 {u'part_number': 5, u'size': 42214217},
 {u'part_number': 6, u'size': 41872731},
 {u'part_number': 7, u'size': 42391120}]

#### Выгружаем данные

In [26]:
tmp_dfs = []
for part_num in map(lambda x: x['part_number'], parts):
    url = '{host}/management/v1/counter/{counter_id}/logrequest/{request_id}/part/{part}/download' \
            .format(
                host=API_HOST,
                counter_id=COUNTER_ID,
                request_id=request_id,
                part=part_num
            )

    r = requests.get(url, headers = headers)
    if r.status_code == 200:
        tmp_df = pd.read_csv(StringIO.StringIO(r.text), sep = '\t')
        tmp_dfs.append(tmp_df)
    else:
        raise ValueError, r.text
        
logs_df = pd.concat(tmp_dfs)

/home/miptgirl/jupyterenv/local/lib/python2.7/site-packages/urllib3/util/ssl_.py:160: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. You can upgrade to a newer version of Python to solve this. For more information, see https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecurePlatformWarning
/home/miptgirl/jupyterenv/local/lib/python2.7/site-packages/urllib3/util/ssl_.py:160: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. You can upgrade to a newer version of Python to solve this. For more information, see https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecurePlatformWarning
/home/miptgirl/jupyterenv/local/lib/python2.7/site-packages/urllib3/util/ssl_.py:160: InsecurePlatformWarning: A

In [28]:
logs_df = logs_df.fillna('')

In [30]:
logs_df.shape[0]

2147596

In [32]:
logs_df.head()

,ym:pv:browser,ym:pv:clientID,ym:pv:date,ym:pv:dateTime,ym:pv:deviceCategory,ym:pv:download,ym:pv:lastTrafficSource,ym:pv:link,ym:pv:notBounce,ym:pv:operatingSystemRoot,ym:pv:params,ym:pv:URL
0,chromemobile,1549964310117886397,2019-02-20,2019-02-20 13:10:14,2,0,organic,0,0,android,,https://ctc.ru/online/
1,chromemobile,1549964310117886397,2019-02-20,2019-02-20 13:10:28,2,0,direct,0,1,android,,https://ctc.ru/online/
2,chrome,1546676248894335702,2019-02-20,2019-02-20 09:03:21,1,0,organic,0,0,windows,,https://ctc.ru/projects/serials/molodezhka/
3,chrome,1546676248894335702,2019-02-20,2019-02-20 09:03:21,1,0,direct,0,0,windows,"{""__ym"":{""adSessionID"":""2977061550642596849""}}",https://ctc.ru/projects/serials/molodezhka/
4,chrome,1546676248894335702,2019-02-20,2019-02-20 09:03:28,1,0,internal,0,0,windows,,https://ctc.ru/projects/serials/molodezhka/vid...


In [38]:
pv_df = logs_df[(logs_df['ym:pv:link'] == 0) & (logs_df['ym:pv:download']  == 0) 
                & (logs_df['ym:pv:notBounce'] == 0) & (logs_df['ym:pv:params'] == '')]

In [39]:
logs_page_views = pv_df.shape[0]
logs_page_views

1559401

### Сравним с ClickHouse

In [40]:
import StringIO
import time
import pandas as pd
import datetime

HOST = 'http://mtgray03k.yandex.ru:8123'
HOST = 'http://mtlog01-01-1.yandex.ru:8123'
#HOST = 'http://clickhouse.metrika.yandex.net:8123'
#HOST = 'http://mtsmart001-1.yandex.ru:8123'
#HO#ST = 'http://mtch01k.metrika.yandex.net:8123'

import requests

def get_clickhouse_data(query, host = HOST, connection_timeout = 1500):
    NUMBER_OF_TRIES = 30
    DELAY = 10
    
    for i in range(NUMBER_OF_TRIES):
        r = requests.post(host, 
                          params = {'timeout_before_checking_execution_speed': 120}, 
                          timeout = connection_timeout, data = query,
                          auth = ('miptgirl', '1QwBgINk')
                         )
        if r.status_code == 200:
            return r.text
        else:
            print 'ATTENTION: try #%d failed' % i
            if i != (NUMBER_OF_TRIES-1):
                #print query
                print r.text
                time.sleep(DELAY*(i+1))
            else:
                raise ValueError, r.text 
        
def get_clickhouse_df(query, host = HOST, connection_timeout = 1500):
    data = get_clickhouse_data(query, host, connection_timeout) 
    df = pd.read_csv(StringIO.StringIO(data), sep = '\t')
    return df

def upload(table, content, host=HOST):
    '''Uploads data to table in ClickHous'''
    content = content.encode('utf-8')
    query_dict = {
             'query': 'INSERT INTO ' + table + ' FORMAT TabSeparatedWithNames '
        }
    r = requests.post(host, data=content, params=query_dict)
    result = r.text
    if r.status_code == 200:
        return result
    else:
        raise ValueError(r.text)

In [44]:
q = '''
select 
    count()
from hits_all
where EventDate = '{date}' and CounterID = {counter}
    and not DontCountHits
format TabSeparatedWithNames
'''.format(counter = COUNTER_ID, date = START_DATE)

get_clickhouse_df(q)

,count()
0,1557874


In [46]:
q = '''
select
    count() as hits,
    DontCountHits,
    IsLink,
    IsArtifical,
    IsDownload,
    IsParameter,
    Refresh,
    IsNotBounce
from hits_all
where EventDate = '{date}' and CounterID = {counter}
group by
    DontCountHits,
    IsLink,
    IsArtifical,
    IsDownload,
    IsParameter,
    Refresh,
    IsNotBounce
format TabSeparatedWithNames
'''.format(counter = COUNTER_ID, date = START_DATE)

get_clickhouse_df(q)

,hits,DontCountHits,IsLink,IsArtifical,IsDownload,IsParameter,Refresh,IsNotBounce
0,961461,0,0,0,0,0,0,0
1,40594,1,0,1,0,0,0,1
2,596413,0,0,1,0,0,0,0
3,1,1,1,0,1,0,0,0
4,541436,1,0,1,0,1,0,0
5,1582,1,0,1,0,0,0,0
6,6109,1,1,0,0,0,0,0


In [47]:
COUNTER_ID

161451